In [1]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 
import matplotlib
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

#Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn import tree

from pandas import DataFrame


# Show progress bar
from tqdm import tqdm

In [2]:
from dtreeviz.trees import dtreeviz # remember to load the package

In [5]:
df = pd.read_csv('flo_dataset_augmented.csv')
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,ligand_amount_mmol,first_sol_amount_ml,second_sol_amount_ml,third_sol_amount_ml,...,x6_zinc stearate,x6_zinc undecylenate,x7_None,x7_copper bromide,x7_oleic acid,x7_water,x7_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,0,0,0,0.154575,-0.011188,-0.247025,-0.673379,-0.370637,-0.096002,...,1,0,1,0,0,0,0,1.750000,480.0,539.0
1,1,1,1,1,0.195883,0.280681,-0.439228,-0.658075,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.610000,610.0,688.5
2,2,2,2,2,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,...,0,0,1,0,0,0,0,3.088889,560.0,595.0
3,3,3,3,3,-0.423738,-0.412509,-0.439228,-0.655198,-0.370637,-0.096002,...,0,0,1,0,0,0,0,3.233333,590.0,635.0
4,4,4,4,4,0.154575,-0.011188,-0.247025,-0.604513,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.383333,482.2,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,210,214,214,214,0.154575,-0.376755,0.713993,4.491519,-0.370637,-0.096002,...,0,0,1,0,0,0,0,4.000000,585.0,630.0
211,211,215,215,215,7.590031,7.504453,-0.439228,5.524499,-0.370637,-0.096002,...,0,0,1,0,0,0,0,2.833333,465.0,550.0
212,212,216,216,216,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,...,0,0,1,0,0,0,0,1.933333,465.0,595.0
213,213,217,217,217,2.633061,0.718486,4.173657,6.213152,4.521657,-0.096002,...,0,0,1,0,0,0,0,3.033333,495.0,595.0


In [6]:
input_col = ['in_amount_mmol', 'p_amount_mmol', 'ligand_amount_mmol',
       'first_sol_amount_ml', 'second_sol_amount_ml', 'third_sol_amount_ml',
       'other_1_amount_mmol', 'other_2_amount_mmol', 'total_volume_ml',
       'temp_c', 'time_min', 'x0_chloroindium oxalate', 'x0_indium acetate',
       'x0_indium bromide', 'x0_indium chloride', 'x0_indium iodide',
       'x0_indium myristate', 'x0_indium oxalate', 'x0_indium palmitate',
       'x0_indium trifluoroacetate',
       'x0_indium tris(N,N-diisopropylacetamidinato)',
       'x1_bis(trimethylsilyl)phosphine', 'x1_phosphine gas',
       'x1_phosphorus trichloride', 'x1_sodium phosphide',
       'x1_tris(diethylamino)phosphine', 'x1_tris(dimethylamino)phosphine',
       'x1_tris(trimethylgermyl)phosphine', 'x1_tris(trimethylsilyl)phosphine',
       'x1_white phosphorus', 'x2_None', 'x2_dodecanethiol', 'x2_lauric acid',
       'x2_myristic acid', 'x2_oleic acid', 'x2_palmitic acid',
       'x2_stearic acid', 'x3_4-ethylpyridine', 'x3_None',
       'x3_dimethylformamide', 'x3_dodecylamine', 'x3_mesitylene',
       'x3_octadecene', 'x3_oleylamine', 'x3_trioctylamine',
       'x3_trioctylphosphine', 'x3_trioctylphosphine oxide', 'x4_None',
       'x4_dioctyl ether', 'x4_dioctylamine', 'x4_hexadecylamine',
       'x4_octylamine', 'x4_oleylamine', 'x4_toluene', 'x4_trioctylphosphine',
       'x4_trioctylphosphine oxide', 'x5_None', 'x5_trioctylphosphine',
       'x6_None', 'x6_acetic acid', 'x6_superhydride',
       'x6_tetrabutylammonium myristate', 'x6_zinc acetate', 'x6_zinc bromide',
       'x6_zinc chloride', 'x6_zinc iodide', 'x6_zinc octanoate',
       'x6_zinc oleate', 'x6_zinc stearate', 'x6_zinc undecylenate', 'x7_None',
       'x7_copper bromide', 'x7_oleic acid', 'x7_water', 'x7_zinc iodide']

output_col = ['diameter_nm', 'abs_nm', 'emission_nm']

X = df[input_col]

Y = df[output_col]

In [7]:
model_SO_diameter = joblib.load('model_SO_diameter_DecisionTree.joblib')
model_SO_emission = joblib.load('model_SO_emission_ExtraTrees.joblib')
model_SO_abs = joblib.load('model_SO_abs_DecisionTree.joblib')

model_MO = joblib.load('model_MO_DecisionTree.joblib')



/home/anhhaokudo/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/anhhaokudo/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/anhhaokudo/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator ExtraTreesRegressor from version 0.24.1 when using version 1.0.2. This might

In [13]:
text_representation = tree.export_text(model_SO_abs)
print(text_representation)

|--- feature_53 <= 0.50
|   |--- feature_5 <= -0.63
|   |   |--- feature_40 <= 0.50
|   |   |   |--- feature_15 <= 0.50
|   |   |   |   |--- feature_42 <= 0.50
|   |   |   |   |   |--- feature_9 <= -0.15
|   |   |   |   |   |   |--- feature_2 <= -0.23
|   |   |   |   |   |   |   |--- feature_58 <= 0.50
|   |   |   |   |   |   |   |   |--- value: [480.00]
|   |   |   |   |   |   |   |--- feature_58 >  0.50
|   |   |   |   |   |   |   |   |--- value: [466.33]
|   |   |   |   |   |   |--- feature_2 >  -0.23
|   |   |   |   |   |   |   |--- value: [436.00]
|   |   |   |   |   |--- feature_9 >  -0.15
|   |   |   |   |   |   |--- value: [500.00]
|   |   |   |   |--- feature_42 >  0.50
|   |   |   |   |   |--- feature_9 <= -0.14
|   |   |   |   |   |   |--- feature_4 <= -0.32
|   |   |   |   |   |   |   |--- value: [460.00]
|   |   |   |   |   |   |--- feature_4 >  -0.32
|   |   |   |   |   |   |   |--- feature_4 <= -0.32
|   |   |   |   |   |   |   |   |--- value: [457.00]
|   |   |   |   | 

In [30]:
s = Y_d.squeeze()
t = X.squeeze()
print(type(s))

<class 'pandas.core.series.Series'>


In [52]:
from dtreeviz.trees import dtreeviz # remember to load the package

viz = dtreeviz(model_SO_diameter, X, s,
                target_name="target",
                feature_names=input_col)

viz.save('dtreeviz.svg')

In [9]:
import graphviz 


In [47]:
dot_data = tree.export_graphviz(model_SO_diameter, out_file=None,
                               feature_names=input_col,
                                filled=True,
                                rounded=True,  
                              special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render("2")
graph.save()

'Gini'

In [10]:
dot_data = tree.export_graphviz(model_MO, out_file=None,
                               feature_names=input_col,
                                filled=True,
                                rounded=True,  
                              special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render("DT_MC")
graph.save("DT_MC")

'DT_MC'

In [11]:
dot_data = tree.export_graphviz(model_SO_diameter, out_file=None,
                               feature_names=input_col,
                                filled=True,
                                rounded=True,  
                              special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render("DT_SC_diameter")
graph.save("DT_SC_diameter")

'DT_SC_diameter'